In [4]:
"""Redirecting writing

If using a library that can print messages to the console, editing the library
by  replacing `print()` with `tqdm.write()` may not be desirable.
In that case, redirecting `sys.stdout` to `tqdm.write()` is an option.

To redirect `sys.stdout`, create a file-like class that will write
any input string to `tqdm.write()`, and supply the arguments
`file=sys.stdout, dynamic_ncols=True`.

A reusable canonical example is given below:
"""
from __future__ import print_function
from time import sleep
import contextlib
import sys
from tqdm import tqdm


class DummyTqdmFile(object):
    """Dummy file-like that will write to tqdm"""
    file = None

    def __init__(self, file):
        self.file = file

    def write(self, x):
        # Avoid print() second call (useless \n)
        if len(x.rstrip()) > 0:
            tqdm.write(x, file=self.file)

    def flush(self):
        return getattr(self.file, "flush", lambda: None)()


@contextlib.contextmanager
def std_out_err_redirect_tqdm():
    orig_out_err = sys.stdout, sys.stderr
    try:
        # sys.stdout = sys.stderr = DummyTqdmFile(orig_out_err[0])
        sys.stdout, sys.stderr = map(DummyTqdmFile, orig_out_err)
        yield orig_out_err[0]
    # Relay exceptions
    except Exception as exc:
        raise exc
    # Always restore sys.stdout/err if necessary
    finally:
        sys.stdout, sys.stderr = orig_out_err


def some_fun(i):
    print("Fee, fi, fo,".split()[i])


# Redirect stdout to tqdm.write()
with std_out_err_redirect_tqdm() as orig_stdout:
    # tqdm needs the original stdout
    # and dynamic_ncols=True to autodetect console width
    for i in tqdm(range(3), file=orig_stdout, dynamic_ncols=True):
        # order of the following two lines should not matter
        some_fun(i)
        sleep(.5)

# After the `with`, printing is restored
print("Done!")


  0%|          | 0/3 [00:00<?, ?it/s]
Fee,

 33%|███▎      | 1/3 [00:00<00:01,  1.93it/s]
fi,

 67%|██████▋   | 2/3 [00:01<00:00,  1.92it/s]
fo,

100%|██████████| 3/3 [00:01<00:00,  1.92it/s]
Done!


In [5]:
"""
# Simple tqdm examples and profiling

# Benchmark
for i in _range(int(1e8)):
    pass

# Basic demo
import tqdm
for i in tqdm.trange(int(1e8)):
    pass

# Some decorations
import tqdm
for i in tqdm.trange(int(1e8), miniters=int(1e6), ascii=True,
                     desc="cool", dynamic_ncols=True):
    pass

# Nested bars
from tqdm import trange
for i in trange(10):
    for j in trange(int(1e7), leave=False, unit_scale=True):
        pass

# Experimental GUI demo
import tqdm
for i in tqdm.tgrange(int(1e8)):
    pass

# Comparison to https://code.google.com/p/python-progressbar/
try:
    from progressbar.progressbar import ProgressBar
except ImportError:
    pass
else:
    for i in ProgressBar()(_range(int(1e8))):
        pass

# Dynamic miniters benchmark
from tqdm import trange
for i in trange(int(1e8), miniters=None, mininterval=0.1, smoothing=0):
    pass

# Fixed miniters benchmark
from tqdm import trange
for i in trange(int(1e8), miniters=4500000, mininterval=0.1, smoothing=0):
    pass
"""

from time import sleep
from timeit import timeit
import re

# Simple demo
from tqdm import trange
for i in trange(16, leave=True):
    sleep(0.1)

# Profiling/overhead tests
stmts = filter(None, re.split(r'\n\s*#.*?\n', __doc__))
for s in stmts:
    print(s.replace('import tqdm\n', ''))
    print(timeit(stmt='try:\n\t_range = xrange'
                 '\nexcept:\n\t_range = range\n' + s, number=1), 'seconds')


100%|██████████| 16/16 [00:01<00:00,  9.58it/s]


for i in _range(int(1e8)):
    pass


  0%|          | 284344/100000000 [00:00<00:35, 2814282.02it/s]

2.137384172999191 seconds
for i in tqdm.trange(int(1e8)):
    pass



 11%|█         | 11169199/100000000 [00:03<00:25, 3461244.45it/s]


 23%|██▎       | 22694890/100000000 [00:06<00:21, 3560016.56it/s]


 34%|███▍      | 34202267/100000000 [00:09<00:18, 3588884.43it/s]


 35%|███▍      | 34939004/100000000 [00:09<00:18, 3590715.74it/s]

KeyboardInterrupt: ignored

In [3]:
import tqdm
for i in tqdm.trange(int(1e8), miniters=int(1e6), ascii=True,
                     desc="cool", dynamic_ncols=True):
    pass


cool:  48%|####8     | 48000000/100000000 [00:11<00:12, 4269544.76it/s]

KeyboardInterrupt: ignored

In [2]:
from time import sleep
# from tqdm import tqdm
import tqdm

pbar = tqdm.tqdm(["a", "b", "c", "d"])
# pbar = tqdm.tqdm_gui(["a", "b", "c", "d"])
for char in pbar:
#     pbar.set_description("Processing %s" % char)
    sleep(0.5)
    i = ord(char)
    pbar.set_postfix({'i': i})

100%|██████████| 4/4 [00:02<00:00,  1.98it/s, i=100]


# in colab notebook

In [0]:
from time import sleep
from tqdm import tqdm_notebook as tqdm

In [0]:
from tqdm import tqdm, trange
tnrange=trange
tqdm_notebook=tqdm

In [7]:
pbar = tqdm(["a", "b", "c", "d"])
# pbar = tqdm.tqdm_gui(["a", "b", "c", "d"])
for char in pbar:
#     pbar.set_description("Processing %s" % char)
    sleep(0.5)
    i = ord(char)
    pbar.set_postfix({'i': i})

100%|██████████| 4/4 [00:02<00:00,  1.99it/s, i=100]
